In [76]:
import pandas as pd
import wandb

api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("2205new_weights-22-05-2024")

summary_list, config_list, name_list = [], [], []
for run in runs:
    summary_list.append(run.summary._json_dict["test_acc"])

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append({k: v for k, v in run.config.items() if not k.startswith("_")})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame(
    {"test_acc": summary_list, "config": config_list, "name": name_list}
)
config_df = pd.json_normalize(runs_df["config"])
df = pd.concat([runs_df.drop(columns=["config"]), config_df], axis=1)
df["data.datasets"] = df["data.datasets"].apply(lambda x: x[0])

In [77]:
import pandas as pd
import wandb

api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("2205forward_transfer-22-05-2024-test_only")

summary_list, config_list, name_list = [], [], []
for run in runs:
    summary_list.append(run.summary._json_dict["test_acc"])

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append({k: v for k, v in run.config.items() if not k.startswith("_")})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame(
    {"test_acc": summary_list, "config": config_list, "name": name_list}
)
config_df = pd.json_normalize(runs_df["config"])
scratch_df = pd.concat([runs_df.drop(columns=["config"]), config_df], axis=1)
# df["data.datasets"] = df["data.datasets"].apply(lambda x: x[0])

In [81]:
def computer_BWT(filtered_df):
    for i in range(1, 5):
        inner_running_sum = 0
        for j in range(i):
            jj = filtered_df[
                (filtered_df["data_task"] == j) & (filtered_df["task"] == j)
            ]["test_acc"].item()
            ij = filtered_df[
                (filtered_df["data_task"] == j) & (filtered_df["task"] == i)
            ]["test_acc"].item()

            inner_running_sum += (ij - jj) / i

    inner_running_sum /= 4
    inner_running_sum *= 100
    return inner_running_sum


def computer_FWT(filtered_df_frozen, df_scratch):
    for i in range(1, 5):
        inner_running_sum = 0
        ii = filtered_df_frozen[
            (filtered_df_frozen["data_task"] == i) & (filtered_df_frozen["task"] == i)
        ]["test_acc"].item()
        ii_dash = df_scratch[(df_scratch["data_task"] == i)]["test_acc"].item()

        inner_running_sum += (ii - ii_dash) / i

    inner_running_sum /= 4
    inner_running_sum *= 100
    return inner_running_sum

In [82]:
from itertools import product


regs = [True, False]
approaches = {
    "finetuning": ("finetuning", 0),
    "replay": ("finetuning", 2000),
    "ewc": ("ewc", 0),
    "lwf": ("lwf", 0),
}
results = []
for is_reg, (real_name, (approach, exemplars)) in product(regs, approaches.items()):
    filtered_df = df[
        (df["training.approach.name"] == approach)
        & (df["data.exemplars.num_exemplars"] == exemplars)
        & (df["training.vcreg.reg_layers"].notna() == is_reg)
    ]

    bwt = computer_BWT(filtered_df)
    fwt = computer_FWT(filtered_df, scratch_df)
    results.append((real_name, is_reg, bwt, fwt))

result_df = pd.DataFrame(
    results,
    columns=[
        "approach",
        "reg_layers",
        "bwt",
        "fwt",
    ],
)


# computer_BWT(filtered_df)

In [84]:
formating = lambda x: "{:.2f}".format(x)
result_df.groupby(["approach", "reg_layers"]).mean()

bwt       fwt
approach   reg_layers                    
ewc        False      -1.300000  0.396875
           True       -0.468750  0.559375
finetuning False      -1.628125  0.506250
           True       -0.978125  0.609375
lwf        False       0.337500  0.187500
           True        0.256250  0.425000
replay     False      -0.175000  0.487500
           True        0.000000  0.643750